## SILVER TO GOLD LAYER


In [15]:
import pandas as pd
from global_paths import GlobalPaths
from common_utilities import TradeHistory

### TradeHistory Paths


In [16]:
# Initialize TradeHistory Paths
TradeHistoryPath = GlobalPaths(source_name="DATA", object_name="TradeHistory")
TradeHistorySilverPath = TradeHistoryPath.createLayer(layer_name="SILVER")
TradeHistorySilverFile = TradeHistorySilverPath.joinpath("TradeHistory_data.csv")
TradeHistoryGoldPath = TradeHistoryPath.createLayer(layer_name="GOLD")
TradeHistoryGoldFile = TradeHistoryGoldPath.joinpath("TradeHistory_data.csv")

# Initialize Equity Paths
EquityPath = GlobalPaths(source_name="DATA", object_name="Equity")
EquitySilverPath = EquityPath.createLayer(layer_name="SILVER")
EquitySilverFile = EquitySilverPath.joinpath("Equity_data.csv")

### Gold Layer - Trade History


In [17]:
# read the csv file
df_TradeHistory = pd.read_csv(TradeHistorySilverFile)  # , convert_dates=False

# # Filter Equity trades
df_TradeHistory = df_TradeHistory[df_TradeHistory["segment"] == "EQ"]

# replace scrip code to compnay name
df_Equity = pd.read_csv(EquitySilverFile)
df_Equity["security_code"] = df_Equity["security_code"].astype(str)

# Merge df_TradeHistory with df_Equity on the matching columns
df_TradeHistory = df_TradeHistory.merge(
    df_Equity[["security_code", "security_id"]],
    left_on="scrip_code",
    right_on="security_code",
    how="left",
)


# Assign the new column 'stock_name' in df_TradeHistory to the values from 'security_id'
df_TradeHistory["stock_name"] = df_TradeHistory["security_id"].combine_first(
    df_TradeHistory["company"]
)


# sort the dataframe by date
df_TradeHistory = df_TradeHistory.sort_values(by="datetime")

data_dict = list()
trade_history = dict()
for _, row in df_TradeHistory.iterrows():
    stock_name = row["stock_name"]
    if stock_name not in trade_history:
        trade_history[stock_name] = TradeHistory(stock_name)

    if str(row["trade_type"]).upper() == "BUY":
        row["buy_price"] = row["trade_price"]
        trade_history[stock_name].trade_price.append(row["trade_price"])
        trade_history[stock_name].trade_quantity.append(row["trade_quantity"])
    elif str(row["trade_type"]).upper() == "SELL":
        row["sell_price"] = row["trade_price"]
        row["buy_price"] = trade_history[stock_name].fifo_sell_calc(
            row["trade_quantity"]
        )
    else:
        raise Exception(f'{row["trade_type"]} was never excepected')

    row["holding_quantity"] = trade_history[stock_name].holding_quantity()
    row["avg_price"] = trade_history[stock_name].calc_avg_price()
    data_dict.append(row)

df_TradeHistory = pd.DataFrame(data_dict)
df_TradeHistory = df_TradeHistory.round(2)
df_TradeHistory = df_TradeHistory[
    [
        "datetime",
        "stock_name",
        "trade_type",
        "trade_quantity",
        "buy_price",
        "sell_price",
        "holding_quantity",
        "avg_price",
    ]
]


# Save the result as a csv file
df_TradeHistory.to_csv(TradeHistoryGoldFile, index=None)
df_TradeHistory.info()
print("Gold Layer csv file for trade history successfully created at:")
print(TradeHistoryGoldFile.resolve())

TypeError: NDFrame.to_csv() got an unexpected keyword argument 'orient'